In [1]:
import nltk
import numpy as np
import pandas as pd
import random
import math
from sklearn.model_selection import train_test_split
import time

In [2]:
# We use treebanck corpus as our dataset.
data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
print("There are total %s sentence in the dataset." %len(data))

#shuffle the data
random.shuffle(data)
random.seed(0)

#splite the data, change N to change training data size.
N = 0.80
train_data = data[: math.floor(len(data)*N)]
test_data = data[math.floor(len(data)*N): ]

#train_test_split(nltk_data,train_size=N,test_size=1-N,random_state = 101)
print("There are %s training data." %len(train_data))
print("There are %s testing data." %len(test_data))

#split training sentence and test sentence into single word.
train_split = [word for sentence in train_data for word in sentence]
test_split = [word for sentence in test_data for word in sentence]

print("There are %s training word." %len(train_split))
print("There are %s testing word." %len(test_split))

There are total 3914 sentence in the dataset.
There are 3131 training data.
There are 783 testing data.
There are 80159 training word.
There are 20517 testing word.


In [3]:
#Transition Probability for specific pair of tags
def transition(front_tag, back_tag, train_split = train_split):
    countfront = 0
    countfollow = 0
    for index, pair in enumerate(train_split):
        if pair[1] == front_tag:
            countfront += 1
            if  index+1 != len(train_split) and train_split[index+1][1] == back_tag:
                countfollow += 1
                
    trans_prob = countfollow/countfront
    return trans_prob


# Emission Probability for specific word 
def Emission(word, tag, train_splite = train_split):
    num_word = len([st[0] for st in train_splite if st[0] == word and st[1] ==tag])
    num_tag = len([st for st in train_split if st[1] == tag])
    return num_word/num_tag

In [4]:
#check how many unqiue tag we have.
tags = np.unique([st[1] for st in train_split])
tag_len = len(tags)

# creat the transition matrix.
trans_matrix = np.zeros((tag_len, tag_len))

for row in range(tag_len):
    for col in range(tag_len):
        trans_matrix[row][col] = transition(tags[row], tags[col])
        
#print out the matrix table
matrix_table = pd.DataFrame(trans_matrix, index=tags, columns = tags)
display(matrix_table)

,.,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRON,PRT,VERB,X
.,0.095412,0.044268,0.090792,0.052326,0.059847,0.174815,0.215322,0.078973,0.065757,0.002579,0.091759,0.028043
ADJ,0.064415,0.065977,0.079641,0.003709,0.016787,0.005661,0.699395,0.020301,0.000781,0.011321,0.011517,0.020496
ADP,0.039501,0.103338,0.016947,0.013507,0.000892,0.330912,0.322885,0.059760,0.069062,0.001656,0.008410,0.033129
ADV,0.135315,0.134142,0.116543,0.081345,0.007039,0.068831,0.031678,0.032069,0.014470,0.013297,0.345327,0.019945
CONJ,0.034292,0.112832,0.054757,0.056416,0.000000,0.112832,0.348451,0.044248,0.063606,0.006084,0.157633,0.008850
DET,0.018466,0.204687,0.009375,0.012642,0.000568,0.005824,0.640341,0.022727,0.003409,0.000142,0.039205,0.042614
NOUN,0.238516,0.012830,0.176380,0.017734,0.042519,0.012830,0.262294,0.009327,0.004554,0.044183,0.149538,0.029295
NUM,0.119099,0.034692,0.037196,0.002861,0.014664,0.003577,0.348712,0.183834,0.001788,0.028255,0.018956,0.206366
PRON,0.040732,0.075515,0.022426,0.032952,0.005034,0.009153,0.208696,0.006865,0.008696,0.013730,0.484211,0.091991
PRT,0.044688,0.088201,0.020384,0.010192,0.002352,0.103489,0.239514,0.058800,0.018032,0.001960,0.398667,0.013720


In [5]:
def Viterbi(words, tags):
    seq = []
    final_seq = []
    for index, word in enumerate(words):
        prob = [] 
        for tag in tags:
            if index == 0:
                tran_prob = matrix_table.loc['.', tag]  
            else:
                tran_prob = matrix_table.loc[seq[-1], tag]

            emi_prob = Emission(word, tag)
            final_prob = emi_prob * tran_prob  
            
            prob.append(final_prob)
            
        most_poss_index = prob.index(max(prob))
        final_max = tags[most_poss_index] 
        seq.append(final_max)
    
    for i in range(len(seq)):
        final_seq.append((words[i], seq[i]))
        
    return final_seq


In [9]:
random.seed(1)

def check_accuracy(test_word, actual_data, tags):
    test_num = len(test_word)
    start = time.time()
    predict = Viterbi(test_word, tags)
    end = time.time()

    print("Totsl time spent: %s" % (end-start))

    right = 0
    for i in range(test_num):
        if predict[i] == actual_data[i]:
            right+=1
    
    return right/test_num*100
    

# select n random sentence, make actual data for checking, and testing 
# print(len(test_data))
# for i in range(10):
#     print(random.randint(1,len(test_data)))
# test_subset = [test_data[random.randint(1,len(test_data))] for i in range(10)]

# for i in range(1,10):
#     actual_data = []
#     test_word = []
#     for sentence in test_subset[:i]:
#         for st in sentence:
#             actual_data.append(st)
#             test_word.append(st[0])
            
#     print('Viterbi Algorithm Accuracy: ',check_accuracy(test_word, actual_data, tags))
    
    
words = ['I', "am", "so", "happy", "to", "see", "you", "!"]
print(Viterbi(words, tags))
#print(check_accuracy(words, actual_data, tags))

[('I', 'PRON'), ('am', 'VERB'), ('so', 'ADV'), ('happy', 'ADJ'), ('to', 'PRT'), ('see', 'VERB'), ('you', 'PRON'), ('!', '.')]


In [14]:
user_sentence = input()
user_word = user_sentence.split(' ')
viterbi = Viterbi(user_word, tags)
for st in viterbi:
    print(st[0],':', st[1])

I am happy
I : PRON
am : VERB
happy : ADJ
